In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    scs_set = set()
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) == len(shortest_sup):
            scs_set.add(sup) # add to scs_set if same lenght is found
        if shortest_sup is None or len(sup) < len(shortest_sup):
            scs_set = set() # reset if shorter scs is found
            scs_set.add(sup)
            shortest_sup = sup  # found shorter superstring
    return shortest_sup, scs_set  # return shortest

Question 1

In a practical, we saw the scs function (copied below along with overlap for finding the shortest common superstring of a set of strings.

It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings ABC, BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.

What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [2]:
setstrings = {"CCT", "CTT", "TGC", "TGG", "GAT", "ATT"}

scs, all_scs = scs(setstrings)
print(len(scs))
print(scs)

11
CCTTGGATTGC


2.
Question 2

How many different shortest common superstrings are there for the input strings given in the previous question?

Hint 1: You can modify the scs function to keep track of this. 

Hint 2: You can look at these examples to double-check that your modified scs is working as expected. 

In [3]:
print(len(all_scs))

4


3.
Question 3

Download this FASTQ file containing synthetic sequencing reads from a mystery virus:

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

In [4]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line: .readline() method reads a single line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences
virus_reads = readFastq("ads1_week4_reads.fq")

In [8]:
# build a k-mer index of the reads
def IndexReads(reads, k):
    k_ind = {}
    for i in range(len(reads)):
        k_set = set()
        n_kmers = len(reads[i])-k+1
        for j in range(n_kmers):
            kmer = reads[i][j:j+(k)]
            k_set.add(reads[i])
            if kmer not in k_ind: # check if key already exists, initiate key:value pair when it doesn't
                k_ind[kmer] = []
            k_ind[kmer].append(reads[i]) # append read to kmer dictionary entry
    return(k_ind)

# function to find the biggest overlap
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    k_ind = IndexReads(reads, k)
    # for all reads, find the two reads with greatest overlap (if tie, first one is returned)
    # modification: use index to reduce number of reads to try overlapping
    for r in reads:
        suffix = r[-k:]
        reads_w_suff = k_ind[suffix]
        for o in reads_w_suff:
            if o != r:
                olen = overlap(r, o, min_length=k)
                if olen > best_olen:
                    reada, readb = r, o
                    best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        # for all overlaps, append a to suffix of b
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
        # if no more overlaps are present (no more edges), concatenate remaining nodes
    return ''.join(reads)

In [9]:
greedy_scs(['ABC', 'BCA', 'CAB'], 2)

'CABCA'

In [10]:
mystery_virus = greedy_scs(virus_reads, 20)
len(mystery_virus)

15894

4.
Question 4

How many Ts are there in the full, assembled genome from the previous question?

In [12]:
mystery_virus.count("T")

3723

In [14]:
mystery_virus[1:500]

'CCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCTAGTACACTTAGGATTCAAGATCCTATTATCAGGGACAAGAGCAGGATTAGGGATATCCGAGATGGCCACACTTTTGAGGAGCTTAGCATTGTTCAAAAGAAACAAGGACAAACCACCCATTACATCAGGATCCGGTGGAGCCATCAGAGGAATCAAACACATTATTATAGTACCAATTCCTGGAGATTCCTCAATTACCACTCGATCCAGACTACTGGACCGGTTGGTCAGGTTAATTGGAAACCCGGATGTGAGCGGGCCCAAACTAACAGGGGCACTAATAGGTATATTATCCTTATTTGTGGAGTCTCCAGGTCAATTGATTCAGAGGATCACCGATGACCCTGACGTTAGCATCAGGCTGTTAGAGGTTGTTCAGAGTGACCAGTCACAATCTGGCCTTACCTTCGCATCAAGAGGTACCAACATGGAGGATGAGGCGGACCAATACTTT'